## Вступ

У цьому уроці розглянемо:
- Що таке виклик функції та для чого він потрібен
- Як створити виклик функції за допомогою Azure OpenAI
- Як інтегрувати виклик функції у ваш додаток

## Навчальні цілі

Після проходження цього уроку ви знатимете та розумітимете:

- Для чого використовувати виклик функції
- Як налаштувати виклик функції через сервіс Azure Open AI
- Як розробити ефективні виклики функцій для вашого додатку


## Розуміння викликів функцій

У цьому уроці ми створюємо функцію для нашого освітнього стартапу, яка дозволяє користувачам знаходити технічні курси за допомогою чат-бота. Ми будемо рекомендувати курси, які відповідають їхньому рівню знань, поточній ролі та цікавій технології.

Для цього ми використаємо комбінацію:
 - `Azure Open AI` для створення чат-досвіду для користувача
 - `Microsoft Learn Catalog API` для допомоги у пошуку курсів на основі запиту користувача
 - `Function Calling` для того, щоб взяти запит користувача і передати його у функцію для виконання API-запиту

Щоб розпочати, давайте розглянемо, навіщо взагалі використовувати виклики функцій:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # отримати нову відповідь від GPT, де він може бачити відповідь функції


print(second_response.choices[0].message)


### Чому варто використовувати Function Calling

Якщо ви вже проходили інші уроки цього курсу, ви, ймовірно, розумієте, наскільки потужними є великі мовні моделі (LLMs). Сподіваємось, ви також помітили деякі їхні обмеження.

Function Calling — це функція Azure Open AI Service, яка допомагає подолати такі обмеження:
1) Послідовний формат відповідей
2) Можливість використовувати дані з інших джерел додатка у чаті

До появи function calling відповіді LLM були неструктурованими та непослідовними. Розробникам доводилося писати складний код для перевірки, щоб обробляти всі можливі варіації відповідей.

Користувачі не могли отримати відповіді на запитання на кшталт "Яка зараз погода у Стокгольмі?". Це тому, що моделі обмежувалися даними, на яких їх навчали.

Розглянемо приклад нижче, який ілюструє цю проблему:

Припустимо, ми хочемо створити базу даних студентів, щоб рекомендувати їм відповідні курси. Нижче наведено два описи студентів, які містять дуже схожі дані.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Ми хочемо надіслати це LLM для обробки даних. Згодом це можна буде використати у нашому застосунку для надсилання до API або зберігання у базі даних.

Давайте створимо два однакових запити, якими ми пояснюємо LLM, яка саме інформація нас цікавить:


Ми хочемо надіслати це LLM, щоб проаналізувати частини, які важливі для нашого продукту. Тому ми можемо створити два ідентичні підказки, щоб дати інструкції LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Після створення цих двох підказок ми надішлемо їх до LLM, використовуючи `openai.ChatCompletion`. Ми зберігаємо підказку у змінній `messages` і призначаємо роль `user`. Це потрібно для імітації повідомлення від користувача, яке надсилається чат-боту.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Навіть якщо підказки однакові, а описи схожі, ми можемо отримати різні формати властивості `Grades`.

Якщо ви запустите вищенаведену комірку кілька разів, формат може бути як `3.7`, так і `3.7 GPA`.

Це відбувається тому, що LLM отримує неструктуровані дані у вигляді текстової підказки і також повертає неструктуровані дані. Нам потрібно мати структурований формат, щоб знати, чого очікувати при збереженні чи використанні цих даних.

Використовуючи функціональні виклики, ми можемо бути впевнені, що отримаємо структуровані дані у відповідь. При використанні функціональних викликів LLM насправді не викликає і не запускає жодних функцій. Натомість ми створюємо структуру, якої LLM має дотримуватися у своїх відповідях. Потім ми використовуємо ці структуровані відповіді, щоб знати, яку функцію запускати у наших застосунках.


![Схема потоку виклику функції](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.uk.png)


### Варіанти використання викликів функцій

**Виклик зовнішніх інструментів**  
Чат-боти чудово відповідають на запитання користувачів. Завдяки виклику функцій, чат-боти можуть використовувати повідомлення від користувачів для виконання певних завдань. Наприклад, студент може попросити чат-бота: "Надішли лист моєму викладачу з проханням допомогти мені з цією темою". Для цього можна викликати функцію `send_email(to: string, body: string)`

**Створення API або запитів до бази даних**  
Користувачі можуть знаходити інформацію, використовуючи природну мову, яка перетворюється у форматований запит або API-запит. Наприклад, вчитель може запитати: "Хто з учнів виконав останнє завдання?", і це може викликати функцію з назвою `get_completed(student_name: string, assignment: int, current_status: string)`

**Створення структурованих даних**  
Користувачі можуть взяти фрагмент тексту або CSV і за допомогою LLM виділити з нього важливу інформацію. Наприклад, студент може перетворити статтю з Вікіпедії про мирні угоди на AI-картки для навчання. Це можна зробити за допомогою функції `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Створення вашого першого виклику функції

Процес створення виклику функції складається з трьох основних кроків:
1. Викликати Chat Completions API зі списком ваших функцій і повідомленням користувача
2. Прочитати відповідь моделі, щоб виконати дію, наприклад, викликати функцію або API
3. Зробити ще один виклик до Chat Completions API з відповіддю від вашої функції, щоб використати цю інформацію для створення відповіді користувачу.


![Потік виклику функції](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.uk.png)


### Елементи виклику функції

#### Введення користувача

Перший крок — створити повідомлення користувача. Це можна зробити динамічно, взявши значення з текстового поля, або ж задати значення тут. Якщо ви вперше працюєте з Chat Completions API, потрібно визначити `role` та `content` повідомлення.

`role` може бути або `system` (створення правил), або `assistant` (модель), або `user` (кінцевий користувач). Для виклику функції ми встановимо це як `user` і додамо приклад запитання.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Створення функцій.

Далі ми визначимо функцію та її параметри. Тут ми використаємо лише одну функцію під назвою `search_courses`, але ви можете створювати кілька функцій.

**Важливо**: Функції додаються до системного повідомлення для LLM і враховуються у загальній кількості доступних вам токенів.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Визначення**

`name` - Назва функції, яку потрібно викликати.

`description` - Опис того, як працює функція. Тут важливо бути конкретним і зрозумілим.

`parameters` - Список значень і формат, які ви хочете отримати у відповіді моделі.

`type` - Тип даних, у якому будуть зберігатися властивості.

`properties` - Список конкретних значень, які модель використає у своїй відповіді.

`name` - Назва властивості, яку модель використає у відформатованій відповіді.

`type` - Тип даних цієї властивості.

`description` - Опис конкретної властивості.

**Необов’язково**

`required` - обов’язкова властивість для завершення виклику функції


### Виклик функції
Після того, як ми визначили функцію, тепер потрібно додати її до виклику Chat Completion API. Для цього ми додаємо `functions` до запиту. У цьому випадку це `functions=functions`.

Також є можливість встановити `function_call` у значення `auto`. Це означає, що ми дозволяємо LLM самостійно вирішити, яку функцію викликати на основі повідомлення користувача, замість того щоб призначати її вручну.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Тепер давайте подивимось на відповідь і побачимо, як вона відформатована:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Ви можете побачити, що викликається функція з певною назвою, і з повідомлення користувача LLM зміг знайти дані для заповнення аргументів функції.


## 3. Інтеграція викликів функцій у додаток.

Після того, як ми протестували відформатовану відповідь від LLM, тепер можемо інтегрувати це у додаток.

### Керування потоком

Щоб інтегрувати це у наш додаток, виконаймо наступні кроки:

Спочатку зробимо запит до сервісів Open AI і збережемо повідомлення у змінній з назвою `response_message`.


In [ ]:
response_message = response.choices[0].message

Тепер ми визначимо функцію, яка викликатиме API Microsoft Learn для отримання списку курсів:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Як найкраща практика, ми спочатку перевіримо, чи модель хоче викликати функцію. Після цього ми створимо одну з доступних функцій і зіставимо її з тією, яку викликає модель.
Далі ми візьмемо аргументи функції та зіставимо їх з аргументами від LLM.

Нарешті, ми додамо повідомлення про виклик функції та значення, які були повернуті повідомленням `search_courses`. Це дає LLM всю необхідну інформацію,
щоб відповісти користувачу природною мовою.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Завдання з коду

Чудова робота! Щоб продовжити вивчення Azure Open AI Function Calling, ви можете створити: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Додайте більше параметрів до функції, які можуть допомогти користувачам знаходити більше курсів. Доступні параметри API можна знайти тут:
 - Створіть ще один виклик функції, який приймає додаткову інформацію від користувача, наприклад, його рідну мову
 - Додайте обробку помилок на випадок, якщо виклик функції та/або API не повертає жодного відповідного курсу



---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, звертаємо вашу увагу, що автоматичний переклад може містити помилки або неточності. Оригінальний документ рідною мовою слід вважати авторитетним джерелом. Для отримання критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильне тлумачення, що виникли внаслідок використання цього перекладу.
